# MLflow Example: Tracking Two Model Training Runs

In [1]:
# === Model Training and Experimentation ===

# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits


In [2]:

# Load dataset
data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Define a function to train a model and return its accuracy
def train_model(n_estimators, max_depth):
    # Train a RandomForestClassifier
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    print(f"Model with n_estimators={n_estimators}, max_depth={max_depth} achieved accuracy={accuracy:.4f}")
    return model, accuracy

In [3]:

# Train models with different hyperparameter configurations
print("Training Model 1...")
model_1, acc_1 = train_model(n_estimators=20, max_depth=5)

Training Model 1...
Model with n_estimators=20, max_depth=5 achieved accuracy=0.9389


In [4]:
print("\nTraining Model 2...")
model_2, acc_2 = train_model(n_estimators=100, max_depth=10)



Training Model 2...
Model with n_estimators=100, max_depth=10 achieved accuracy=0.9722


Now adding MLFlow

In [5]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits

In [6]:
import os

mlflow.set_tracking_uri("./mlruns")

In [7]:
# get this code from dagshub
import dagshub
dagshub.init(repo_owner='luna-vyer', repo_name='mlflow-dagshub-pipeline', mlflow=True)

Accessing as luna-vyer

Initialized MLflow to track repo "luna-vyer/mlflow-dagshub-pipeline"

Repository luna-vyer/mlflow-dagshub-pipeline initialized!

In [8]:
data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [9]:
# Set MLflow experiment name. This will get created if it doesn't exist
experiment_name = "RandomForestExperiment_2"
mlflow.set_experiment(experiment_name)


<Experiment: artifact_location='file:///c:/Users/lunav/Documents/GitHub/mlflow-dagshub-pipeline/mlruns/442887814972147372', creation_time=1750863252125, experiment_id='442887814972147372', last_update_time=1750863252125, lifecycle_stage='active', name='RandomForestExperiment_2', tags={}>

# 2.

In [10]:
# Define a function to train a model, log parameters and metrics to MLflow
def train_and_log_model(n_estimators, max_depth):
    with mlflow.start_run() as run: # <--
        # Train a RandomForestClassifier
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Make predictions
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)

        # Log parameters, metrics, and the model itself
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(model, "random_forest_model_2")

        print("Run ID:", run.info.run_id)
    

        print(f"Logged RandomForest model with n_estimators={n_estimators}, max_depth={max_depth}, accuracy={accuracy:.4f}")


In [11]:
# Train and log Model 1
print("Training and Logging Model 1...")
train_and_log_model(n_estimators=20, max_depth=5)

Training and Logging Model 1...


2025/06/30 14:55:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/30 14:55:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: 03573c923f834dbab04428f3d162bb6e
Logged RandomForest model with n_estimators=20, max_depth=5, accuracy=0.9389


In [12]:
# Train and log Model 2
print("\nTraining and Logging Model 2...")
train_and_log_model(n_estimators=100, max_depth=10)


Training and Logging Model 2...


2025/06/30 14:55:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/30 14:55:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: b7026a547ef64f4f946740aa91310103
Logged RandomForest model with n_estimators=100, max_depth=10, accuracy=0.9722


In [13]:
# Instructions to visualize results
print("\nTo view the results, run the following command in your terminal:")
print("mlflow ui")
print("Then navigate to http://127.0.0.1:5000 to explore the experiment results.")


To view the results, run the following command in your terminal:
mlflow ui
Then navigate to http://127.0.0.1:5000 to explore the experiment results.


In [14]:
import joblib

In [15]:
model = RandomForestClassifier(n_estimators=100, max_depth=10)
model.fit(X_train, y_train)

joblib.dump(model, "random_forest_model.pkl")

['random_forest_model.pkl']